In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import transformers
from transformers import pipeline
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import numpy as np
import nltk.data
from requests_html import HTMLSession

In [75]:
url = 'https://ca.finance.yahoo.com/crypto'
response = requests.get(url)
page_content = response.text
doc = BeautifulSoup(page_content, 'html.parser')

# Sentiment analysis on news header

In [3]:
# Model for sentiment analysis
topic_model = "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
label = pipeline('sentiment-analysis', model = topic_model)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [4]:
header = []
news_elements = doc.find_all('a', {'class': "js-content-viewer"})
for i in range (len(news_elements)):
    header.append(news_elements[i].text)
news_header_df = pd.DataFrame({'News_Header': header})

In [5]:
classification = []
for i in range (len(news_header_df.News_Header)):
    classification.append(label(news_header_df.News_Header[i]))

In [9]:
labels = []
for i in range (len(classification)):
    labels.append(classification[i][0]['label'])
news_header_df['sentiment_analysis'] = labels

In [24]:
news_header_df.style.applymap(lambda x: f"background: {'red' if x == 'negative' else 'green' if x == 'positive' else 'non'}", subset=['sentiment_analysis'])

,News_Header,sentiment_analysis
0,MARKETS LIVE BLOGSTOCKS SET TO END BRUTAL WEEK WITH MORE STEEP LOSSESSentiment dampened by China woes amid Evergrande's bankruptcy filing in U.S. court,negative
1,Could Pomerdoge (POMD) New P2E Crypto Meme Overtake Shiba Inu (SHIB) And Dogecoin (DOGE) In 2024?,neutral
2,Dow Jones Futures Slide 200 Points As Stock Market Weakness Continues; Bitcoin Dives On Elon Musk Sales,negative
3,"Bitcoin Price Falls, Then Stabilizes",positive
4,XRP erases gains from Ripple’s partial victory as Bitcoin falls 10% in a week,negative
5,"Bitcoin Calm Shatters With Sudden Tumble, Mass Liquidations",negative
6,Bitcoin Could Fall Another 25%. Will Elon Musk and SpaceX Be to Blame?,negative
7,Bitcoin price heads higher as US CPI data continues to cool,positive
8,UK watchdog raids 34 sites in crypto ATM crackdown,neutral
9,Bitcoin rally falters as crypto pulls back yet again,negative


# Web scrape market data

In [84]:
stock_elements = doc.find('table', {'class' : 'W(100%)'})

In [95]:
stockData=[]
tableTR = stock_elements.find_all('tr')

for i in range (1, len(tableTR)):
    tableTD = tableTR[i].find_all('td')
    name = tableTD[1].text
    price = tableTD[2].text
    change = tableTD[4].text
    volume = tableTD[5].text
    stockData.append([symbol, name, price, change, volume])

columns = ["Symbol", "Name", "Price", "Up/Down", "Volume"]
stockData_df = pd.DataFrame(stockData, columns=columns)
stockData_df

,Symbol,Name,Price,Up/Down,Volume
0,BTC-CAD,Bitcoin CAD,"35,198.25",-6.95%,684.993B
1,ETH-CAD,Ethereum CAD,"2,250.41",-4.56%,270.17B
2,USDT-CAD,Tether USDt CAD,1.3548,+0.02%,112.238B
3,BNB-CAD,BNB CAD,294.17,-3.72%,45.259B
4,XRP-CAD,XRP CAD,0.683184,-12.53%,36.101B
5,USDC-CAD,USD Coin CAD,1.3557,-0.03%,35.554B
6,ADA-CAD,Cardano CAD,0.356469,-2.31%,12.494B
7,DOGE-CAD,Dogecoin CAD,0.084904,-4.44%,11.941B
8,SOL-CAD,Solana CAD,29.30,-5.50%,11.938B
9,TRX-CAD,TRON CAD,0.098939,-2.06%,8.849B


In [87]:
tableTR[0].find_all('td')

[]

In [ ]:
# Assuming the data you're interested in is contained within a table with class "W(100%) M(0)".
table = soup.find("table", class_="W(100%) M(0)")

# Extract the table rows (ignoring the header row if present).
rows = table.find_all("tr")[1:]  # Skip the header row

# Initialize empty lists to store the data.
data = []

# Loop through each row and extract the data from columns.
for row in rows:
    columns = row.find_all("td")
    symbol = columns[0].text
    name = columns[1].text
    price = columns[2].text
    percent_change = columns[3].text
    volume = columns[4].text

    # Append the data to the list.
    data.append([symbol, name, price, percent_change, volume])

# Create a DataFrame from the collected data.
columns = ["Symbol", "Name", "Price", "Percent Change", "Volume"]
df = pd.DataFrame(data, columns=columns)